# Climate Mode Calculation
- calculates climate mode index across CMIP6 models for intercomparison of yield response for phases

## Imports

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob as glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xesmf as xe
import dask

## Processing SST (tos) data from the 18 underlying LOCA2 models
- need to regrid to common grid, everything is in .gn, not .gr

### Inputs

In [16]:
base_path = "/storage/group/pches/default/users/cta5244/CMIP6_tos/ssp370_omon_tos"
era_path = "/storage/group/pches/default/users/cta5244/era5_jan1_2000_tos.nc"
#save_out_path = "/storage/group/pches/default/users/cta5244/CMIP6_tos_rg"
save_out_path = "/storage/work/cta5244/cmip6_tos_gr"
model_paths = sorted(glob.glob(f"{base_path}/*"))
era_ds = xr.open_dataset(era_path)

In [17]:
yearly_paths = sorted(glob.glob(f"{model_paths[0]}/*"))
ds_first = xr.open_mfdataset(yearly_paths)

### Computing

In [3]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="20GiB",
    walltime="02:00:00",
)

cluster.scale(jobs=5) 

Task exception was never retrieved
future: <Task finished name='Task-103540' coro=<Client._gather.<locals>.wait() done, defined at /storage/work/cta5244/.conda/envs/regrid_env/lib/python3.13/site-packages/distributed/client.py:2384> exception=AllExit()>
Traceback (most recent call last):
  File "/storage/work/cta5244/.conda/envs/regrid_env/lib/python3.13/site-packages/distributed/client.py", line 2393, in wait
    raise AllExit()
distributed.client.AllExit


In [4]:
from dask.distributed import Client

client = Client(cluster)

In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://146.186.150.12:8787/status,
Dashboard: http://146.186.150.12:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://146.186.150.12:44449,Workers: 0
Dashboard: http://146.186.150.12:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [42]:
def save_regridded_model(input_model, model_path, save_out_path):
    """
    given model_path, regrids to era5, and saves to some location 
    inputs 
    - model_path
    - save_out_path
    """
    yearly_paths = sorted(glob.glob(f"{model_path}/*"))
    # output string with gr not gn 
    new_list = ['gr' if item == 'gn' else item for item in yearly_paths[0].split("/")[-1].split("_")]
    output_unique_str = "_".join(new_list)
    # opening & regridding datasets
    ds_cmip6 = xr.open_mfdataset(yearly_paths)
    ds_cmip6.load()
    if 'lat' in ds_cmip6.coords:
        ds_cmip6 = ds_cmip6.rename({'lat':'latitude'})
        ds_cmip6 = ds_cmip6.rename({'lon':'longitude'})
    regridder = xe.Regridder(ds_cmip6, era_ds, 'conservative', periodic=True)
    ds_regridded = regridder(ds_cmip6, keep_attrs = True)
    ds_regridded.to_netcdf(f"{save_out_path}/{output_unique_str}.nc")
    return


In [43]:
results = []
for model_path in model_paths:
    out = dask.delayed(save_regridded_model)(input_model = ds_first, model_path=model_path, save_out_path=save_out_path)
    results.append(out)

In [44]:
results = dask.compute(*results)

MemoryError: Unable to allocate 102. GiB for an array with shape (1032, 830305, 16) and data type float64

In [25]:
yearly_paths = sorted(glob.glob(f"{model_paths[2]}/*"))
ds_first = xr.open_mfdataset(yearly_paths)

/tmp/ipykernel_737347/2388071572.py:2: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  ds_first = xr.open_mfdataset(yearly_paths)


In [41]:
ds_first

<xarray.Dataset> Size: 223GB
Dimensions:    (time: 1032, bnds: 2, ncells: 830305, vertices: 16)
Coordinates:
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
    latitude   (ncells) float64 7MB dask.array<chunksize=(415153,), meta=np.ndarray>
    longitude  (ncells) float64 7MB dask.array<chunksize=(415153,), meta=np.ndarray>
Dimensions without coordinates: bnds, ncells, vertices
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 17kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    tos        (time, ncells) float32 3GB dask.array<chunksize=(1, 830305), meta=np.ndarray>
    lat_bnds   (time, ncells, vertices) float64 110GB dask.array<chunksize=(72, 138385, 2), meta=np.ndarray>
    lon_bnds   (time, ncells, vertices) float64 110GB dask.array<chunksize=(72, 138385, 2), meta=np.ndarray>
Attributes: (12/39)
    frequency:              mon
    Conventions:            CF-1.7 CMIP-6.2
    creation_date:          2018-12-18T12:00:00Z
    data_specs_version:     01.00.27
    experiment:             ssp370
    experiment_id:          ssp370
    ...                     ...
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       AWI-CM-1-1-MR
    parent_time_units:      days since 1850-1-1
    parent_variant_label:   r1i1p1f1
    activity_id:            ScenarioMIP AerChemMIP